# Example of querying the UCS HCL using the Server Pivot

## Overview

* Walk the API to get all the arguments for the final query
* The database is optimized to prevent deadend queries using a pruned tree structure
* The tree IDs are used to find the data for the search arguments
* Tree IDs can change as the HCL DB is updated, so they can't be cached

In [1]:
import requests
import json

### Switch the next cell from Markdown to Code for Request debugging when necessary

import logging
import httplib as http_client
http_client.HTTPConnection.debuglevel = 1

logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = True

### Current API location

In [2]:
hcl_root = "https://ucshcltool.cloudapps.cisco.com/public/rest/"

### We start by retrieving the supported server types

#### Target the specific API; takes no parameters

In [3]:
loadServerTypes = "server/loadServerTypes"

In [4]:
resp = requests.post(hcl_root + loadServerTypes)

In [5]:
resp.json()

[{u'ID': 0,
  u'MANAGED': u'UCSM      ',
  u'RELEASE': u'B-Series',
  u'TYPE': u'B',
  u'T_ID': 1},
 {u'ID': 0,
  u'MANAGED': u'UCSM      ',
  u'RELEASE': u'C-Series',
  u'TYPE': u'C',
  u'T_ID': 2},
 {u'ID': 1,
  u'MANAGED': u'Standalone',
  u'RELEASE': u'C-Series',
  u'TYPE': u'C',
  u'T_ID': 4},
 {u'ID': 2,
  u'MANAGED': u'UCSM      ',
  u'RELEASE': u'M-Series',
  u'TYPE': u'M',
  u'T_ID': 3},
 {u'ID': 3,
  u'MANAGED': u'UCSM      ',
  u'RELEASE': u'HX-Series',
  u'TYPE': u'HX',
  u'T_ID': 5}]

#### You can save things away if needed

In [6]:
serverTypes = resp.json()

### Find the B-Series servers

In [7]:
for t in serverTypes:
    if t['TYPE'] == 'B':
        bladeRelease = t
        break
    else:
        raise Exception('Blade type not found in loadServerTypes response')

In [8]:
bladeRelease

{u'ID': 0,
 u'MANAGED': u'UCSM      ',
 u'RELEASE': u'B-Series',
 u'TYPE': u'B',
 u'T_ID': 1}

##### The APIs use the treeId for the parameters of the subsequent requests

In [9]:
sTypeTreeId = { 'treeIdRelease' : str(bladeRelease['T_ID']) }

In [10]:
loadServerModels = "server/loadServerModels"

### The form data is NOT json!!!

In [11]:
resp = requests.post(hcl_root + loadServerModels, data=sTypeTreeId)

In [12]:
resp

<Response [200]>

In [13]:
resp.json()

[{u'ID': u'66,67',
  u'SERVER_MODEL': u'Cisco UCS Scalable M4 Blade Module for UCS B260 M4 or B460 M4 Blade Server with E7 v2 CPU',
  u'T_ID': u'26,27'},
 {u'ID': u'9,42', u'SERVER_MODEL': u'Cisco UCS B22 M3', u'T_ID': u'14,15'},
 {u'ID': u'10,43', u'SERVER_MODEL': u'Cisco UCS B420 M3', u'T_ID': u'20,21'},
 {u'ID': u'70,187', u'SERVER_MODEL': u'Cisco UCS B420 M4', u'T_ID': u'22,23'},
 {u'ID': u'188,189',
  u'SERVER_MODEL': u'3rd Generation Cisco UCS Scalable M4 Blade Module',
  u'T_ID': u'6,7'},
 {u'ID': u'68,69',
  u'SERVER_MODEL': u'Cisco UCS Scalable M4 Blade Module for UCS B260 M4 or B460 M4 Blade Server with E7 v3 CPU',
  u'T_ID': u'28,29'},
 {u'ID': u'4', u'SERVER_MODEL': u'Cisco UCS B230 M1', u'T_ID': u'16'},
 {u'ID': u'5', u'SERVER_MODEL': u'Cisco UCS B230 M2', u'T_ID': u'17'},
 {u'ID': u'1', u'SERVER_MODEL': u'Cisco UCS B250 M2', u'T_ID': u'19'},
 {u'ID': u'3', u'SERVER_MODEL': u'Cisco UCS B250 M1', u'T_ID': u'18'},
 {u'ID': u'7', u'SERVER_MODEL': u'Cisco UCS B440 M2', u'T_ID'

### We want the B200 M4

In [14]:
for s in resp.json():
    if "B200 M4" in s['SERVER_MODEL']:
        blade=s
        break
else:
    raise Exception('Server model B200 M4 not found in loadServerModels')

In [15]:
blade

{u'ID': u'51,157', u'SERVER_MODEL': u'Cisco UCS B200 M4', u'T_ID': u'12,13'}

### Find and select the processors in our blade

In [16]:
sModelTreeId = { 'treeIdServerModel' : blade['T_ID'] }

In [17]:
loadProcessors = "server/loadProcessors"

In [18]:
sModelTreeId

{'treeIdServerModel': u'12,13'}

In [19]:
resp = requests.post(hcl_root + loadProcessors, data=sModelTreeId)

In [20]:
resp

<Response [200]>

In [21]:
resp.json()

[{u'ID': u'61',
  u'PROCESSOR': u'Intel Xeon E5-2600 v3 Series processors',
  u'T_ID': u'153'},
 {u'ID': u'84',
  u'PROCESSOR': u'Intel Xeon E5-2600 v4 Series processors',
  u'T_ID': u'132'}]

### We're using Broadwell (v4) procs

In [22]:
for p in resp.json():
    if "E5-2600 v4" in p['PROCESSOR']:
        proc=p
        break
else:
    raise Exception('Processor model E5-2600 v4 not found in loadProcessors')

In [23]:
p

{u'ID': u'84',
 u'PROCESSOR': u'Intel Xeon E5-2600 v4 Series processors',
 u'T_ID': u'132'}

In [24]:
pModelTreeId = { 'treeIdProcessor' : p['T_ID'] }

In [25]:
pModelTreeId

{'treeIdProcessor': u'132'}

### Now we need to select the OS we're using

In [26]:
loadOsVendors = 'server/loadOsVendors'

In [27]:
resp = requests.post(hcl_root + loadOsVendors, data=pModelTreeId)

In [28]:
resp.json()

[{u'ID': u'4', u'OSVENDOR': u'Red Hat', u'T_ID': u'341'},
 {u'ID': u'8', u'OSVENDOR': u'CentOS', u'T_ID': u'337'},
 {u'ID': u'7', u'OSVENDOR': u'Ubuntu', u'T_ID': u'343'},
 {u'ID': u'3', u'OSVENDOR': u'Oracle', u'T_ID': u'340'},
 {u'ID': u'1', u'OSVENDOR': u'Microsoft', u'T_ID': u'339'},
 {u'ID': u'0', u'OSVENDOR': u'Citrix', u'T_ID': u'338'},
 {u'ID': u'5', u'OSVENDOR': u'VMware', u'T_ID': u'344'},
 {u'ID': u'2', u'OSVENDOR': u'SuSE', u'T_ID': u'342'}]

### VMware is our thing

In [29]:
for osv in resp.json():
    if "VMware" in osv['OSVENDOR']:
        vendor=osv
        break
else:
    raise Exception('Vendor VMware not found in loadOsVendors')

In [30]:
vendor

{u'ID': u'5', u'OSVENDOR': u'VMware', u'T_ID': u'344'}

In [31]:
osVendorTreeId = { 'treeIdVendor' : vendor['T_ID'] }

### And finally find the OS Version

In [32]:
loadOsVersions = 'server/loadOsVersions'

In [33]:
resp = requests.post(hcl_root + loadOsVersions, data=osVendorTreeId)

In [34]:
resp.json()

[{u'ID': u'624', u'OSVERSION': u'vSphere 6.5', u'T_ID': u'2032'},
 {u'ID': u'289', u'OSVERSION': u'vSphere 5.5 U3', u'T_ID': u'2028'},
 {u'ID': u'414', u'OSVERSION': u'vSphere 6.0 U2', u'T_ID': u'2030'},
 {u'ID': u'647', u'OSVERSION': u'vSphere 6.0 U3', u'T_ID': u'2031'},
 {u'ID': u'287', u'OSVERSION': u'vSphere 6.0 U1', u'T_ID': u'2029'}]

In [35]:
for v in resp.json():
    if "6.5" in v['OSVERSION']:
        vers=v
        break
else:
    raise Exception('vSphere 6.5 not found in loadOsVersions')

In [36]:
vers

{u'ID': u'624', u'OSVERSION': u'vSphere 6.5', u'T_ID': u'2032'}

### We have everything we need to submit our query, form the request payload

In [37]:
query = { 'serverType_ID' : str(bladeRelease['ID']),
          'serverModel_ID' : blade['ID'],
          'processor_ID' : proc['ID'],
          'osVendor_ID' : vendor['ID'],
          'osVersion_ID' : vers['ID'],
          'firmwareVersion_ID' : str(-1),
          'manageType' : bladeRelease['MANAGED'].strip() }

In [38]:
query

{'firmwareVersion_ID': '-1',
 'manageType': u'UCSM',
 'osVendor_ID': u'5',
 'osVersion_ID': u'624',
 'processor_ID': u'84',
 'serverModel_ID': u'51,157',
 'serverType_ID': '0'}

In [39]:
loadSearchResults = 'server/loadSearchResults'

In [40]:
resp = requests.post(hcl_root + loadSearchResults, data=query)

In [41]:
resp

<Response [200]>

We now have the results of the query, so we can perform magical things with it, like:
    * Generate a report to be kept in a runbook
    * Compare the driver list to what we're running to make sure we're compliant
    * Check a new FW or OS version against current drivers to see what the impact is

We're just going to dump the FW versions compatible with our config.

Try dumping the full json and experiment around with it

In [42]:
for result in resp.json() :
    print "Version: " + result['Version']

Version: 2.2(8)
Version: 3.1(2)
